In [1]:
import os,glob
from dateutil import parser
import datetime
from IPython.display import clear_output
from ebooklib import epub
import ebooklib

In [2]:
path="/media/some/sophie-hhd/books/epub-cm/"
tp="/media/some/sophie-hhd/books/tmpadd/"
rt=datetime.datetime(2016, 5, 10, 0, 0, 0, 0)
et=datetime.datetime(2016, 5, 11, 0, 0, 0, 0)

In [ ]:
#move files last accessed time betweeen rt and et above
def movFile(rt,et):
    os.chdir(path)
    lst=glob.glob("*.epub")
    os.chdir("/home/some")
    cnt=0
    for f in lst:
        ff=path+f
        atm=! stat $ff |grep "Access: 2"
        atime=parser.parse(atm[0].replace("Access: ","").replace(' +0100',''))
        #if file accessed after 18:00 on 10/05/2016
        if atime >= rt and atime <= et:
            cnt+=1
            print cnt,'-', atime," moving .. ",f

            if cnt % 10 == 0:
                clear_output()
            tf=tp+f
            ! rm $ff
    print cnt       

In [ ]:
parser.parse(atm[0].replace('Access: ',''))

In [ ]:
atm

In [ ]:
#change all files in directory to lowercase
path="/media/some/sophie-hhd/books/epub-m/"
os.chdir(path)
for filename in glob.glob("*.epub"):
    os.rename(filename, filename.lower())
os.chdir("/home/some")

In [ ]:
#reverse the order of author-title to title-author and vice versa
path="/media/some/sophie-hhd/books/epub-cm/"
os.chdir(path)
for filename in glob.glob("*.epub"):
    tmp=filename[:-5].split('-')
    os.rename(filename, '-'.join(t for t in reversed(tmp))+".epub")
os.chdir("/home/some")

### reverse the order of author-title  if auther contains any the,of and change metadata

In [4]:
import pickle
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [5]:
path="/media/some/sophie-hhd/books/epub-cm/"
tgp='/media/some/sophie-hhd/books/exchanged/'

os.chdir(path)
m=glob.glob("*.epub")
os.chdir("/home/some")
try:
    metalst=load_obj('metalst')
except:
    metalst={}
    save_obj(metalst, 'metalst')


In [ ]:
#in file move2
'''metalst=load_obj('metalst')

for mm in m:
    ff=path+mm
    if mm not in metalst.keys():
        tm=!ebook-meta $ff
        metalst[mm]=tm
    
    m.pop(m.index(mm))
    cnt+=1
    if cnt%100==0:
        print cnt
        break
save_obj(metalst, 'metalst')'''

In [6]:

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [106]:
metalst=load_obj('metalst')
#reverse the order of author-title  if auther contains any the,of and change metadata
strlist=['wedding', 'billionaire','wife ','yesterday',' marrying','bedded','millionaire',
         'seal ','winning','mistress','husband','tycoon',' boss',
         'greek','italian','cooking','marriage',' bride',
        'secretary',' nany ','sicilian','manhunt','highland','cowboy','texas','convenient','spaniard'
        ' virgin','forever','bought','claimed','hunted','wedded','death','ruthless']

cnt=0
for f in metalst.keys():
    ff=path+f
    try:
        #get metadata

        metadataR=metalst[f]
        #convert to dictionary
        metadata={}
        for met in metadataR:
            tmp=met.split(":")
            if len(tmp) >= 2:
                metadata[tmp[0].strip()]=tmp[1].strip()
            elif len(tmp)==1:
                metadata[tmp[0].strip()]='Unkown'

        if 'Author(s)' in metadata.keys():

            if any(x in  metadata['Author(s)'].lower(
                ) for x in strlist) or metadata['Author(s)'].lower(
            )[:4]=='the ': #or hasNumbers(metadata['Author(s)']):
                book=epub.read_epub(ff)
                if 'creator' in book.metadata['http://purl.org/dc/elements/1.1/'].keys():
                    book.metadata['http://purl.org/dc/elements/1.1/'].pop('creator')
                book.add_author(metadata['Title'])
                if 'title' in book.metadata['http://purl.org/dc/elements/1.1/'].keys():
                    book.metadata['http://purl.org/dc/elements/1.1/'].pop('title')
                book.set_title(metadata['Author(s)'])
                tit=''.join(ch for ch in book.get_metadata("DC", "title")[0][0] if ch.isalnum())
                auth=''.join(ch for ch in book.get_metadata("DC","creator")[0][0] if ch.isalnum())
                name=auth+"-"+tit+".epub"
                if not os.path.isfile(path+name) and not name == f :
                    name=name.lower()
    
                    print 'saving ..',ff," as .." ,name
                    epub.write_epub(path+name,book,{} )
                    tf=tgp+name.lower()
                    if not os.path.isfile(tf):
                        ! mv $ff $tf
                    else:
                        os.remove(ff)

            if metadata['Author(s)'].count(',') == 1 and metadata['Author(s)'].count('[') == 0:
                book=epub.read_epub(ff)
                if 'creator' in book.metadata['http://purl.org/dc/elements/1.1/'].keys():
                    book.metadata['http://purl.org/dc/elements/1.1/'].pop('creator')
                tmp=metadata['Author(s)'].split(',')
                auth=' '.join(t for t in reversed(tmp))
                book.add_author(auth)
                tit=''.join(ch for ch in book.get_metadata("DC", "title")[0][0] if ch.isalnum())
                auth=''.join(ch for ch in book.get_metadata("DC","creator")[0][0] if ch.isalnum())
                name=auth+"-"+tit+".epub"
                if not os.path.isfile(path+name) and not name == f:
                    name=name.lower()
                    print 'saving ..',ff," as .." ,name
                    epub.write_epub(path+name,book,{} )
                    tf=tgp+name.lower()
                    if not os.path.isfile(tf):
                        ! mv $ff $tf
                    else:
                        os.remove(ff)
                #metalst.pop(f, None)

    except:
        
        #print cnt," failed to read epub removing from dict",f
        metalst.pop(f, None)
        
        #os.remove(ff)
    #tmp=filename[:-5].split('-')
    #os.rename(filename, '-'.join(t for t in reversed(tmp))+".epub")
    cnt+=1
    #if cnt>1:
     #   break
save_obj(metalst, 'metalst')

In [107]:
len(metalst)


4557

In [ ]:
st='yuletideprotector-juliemiller.epub'
tmp=st[:-5].split('-')

st_r='-'.join(t for t in reversed(tmp))+".epub"
st_r

In [ ]:
#remove files in directory path which atime is between rt and et
os.chdir(path)
m=glob.glob("*.epub")
os.chdir("/home/some")
cnt=0
for f in m:
    ff=path+f
    atm=! stat $ff |grep "Access: 2"
    atime=parser.parse(atm[0].replace("Access: ","").replace(' +0100',''))
    #if file accessed after 18:00 on 10/05/2016
    if atime >= rt and atime <= et:
        cnt+=1
        print cnt,'-', atime," moving .. ",f
        
        if cnt % 10 == 0:
            clear_output()
        
        os.remove(ff)
print cnt       


In [ ]:
tf

In [ ]:
os.chdir('/home/')
os.path.isfile('/home/some/author.txt')